# Running a 5-Point Calibration Scan Without OSO Using BITE

This notebook adapts the [Five Point Calibration Scan Controls](https://gitlab.com/ska-telescope/ska-jupyter-scripting/-/blob/main/notebooks/observing/MID_five_point_calibration_scan_controls.ipynb?ref_type=heads) notebook to work with PSI. Running this notebook requires a PSI namespace to be spun up with the CI argument `TMC_ENABLED` set to `true`.

## 1 Setup

### 1.1 Environment Setup

Start by importing all the libraries we'll be using for this notebook.

In [1]:
import json
import os
import random
from datetime import date
from time import sleep

from tango import DevFailed, DeviceProxy

Until notebook clean up is merged need to use this for waiting for state

In [2]:
spinner = ["⣾", "⣽", "⣻", "⢿", "⡿", "⣟", "⣯", "⣷"]


def wait_for_state(device: DeviceProxy, desired_state, break_on_error=True) -> None:
    """Poll a tango device until either the given observation state is reached, or it throws an error.
    Arguments:
    device -- Tango Device to check
    desired_state -- The state which to break upon getting (number or state)
    break_on_error -- If set to False, will keeping running when getting an error status.
    """
    spinL = 0
    poll = 1
    while device.obsState != desired_state:
        if spinL < len(spinner) - 1:
            spinL += 1
        else:
            spinL = 0
        sleep(0.5)
        print(
            "\r",
            f"{spinner[spinL]} Poll# {poll}: Current state is {device.obsState.name}, waiting for {desired_state}...",
            end="",
        )
        if device.obsState == 9 and break_on_error:
            break
        poll += 1
    print(f"\nFinished with: {device.obsState.name}")

### 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [3]:
!kubectl get ns | grep ska-mid-psi

ci-ska-mid-psi-1365379550-alexschell                  Active   3m29s
ci-ska-mid-psi-1365379550-alexschell-sdp              Active   4m11s


And load it into the variables for the notebook:

In [4]:
namespace = "ci-ska-mid-psi-1365379550-alexschell"  # set to desired NS
simulation_mode = 0  # set to 1 to run in sim mode
target_boards_list = [2]  # assign boards
delay_model_filename = "delay_model.json"  # delay model file to use
test_id = "talon2 basic gaussian noise"  # Test to send config from
server = "ska-sdp-kafka." + namespace + ".svc.cluster.local:9092"

Next, we load all the other vars the notebook will use. These should not need to be changed for this run.

In [5]:
# Fully Qualified Domain Names for the devices to set up proxies
CSP_CONTROLLER_FQDN = "mid-csp/control/0"
CSP_SUBARRAY_FQDN = "mid-csp/subarray/01"

CENTRAL_NODE_FQDN = "ska_mid/tm_central/central_node"
LEAF_NODE_FQDN = "ska_mid/tm_leaf_node/csp_master"
LEAF_NODE_SUBARRAY_FQDN = "ska_mid/tm_leaf_node/csp_subarray01"
TMC_SUBARRAY_FQDN = "ska_mid/tm_subarray_node/1"

CBF_CONTROLLER_FQDN = "mid_csp_cbf/sub_elt/controller"
CBF_SUBARRAY_FQDN = "mid_csp_cbf/sub_elt/subarray_01"

BITE_FQDN = "mid_csp_cbf/ec/bite"
DEPLOYER_FQDN = "mid_csp_cbf/ec/deployer"

DELAY_MODEL_FQDN = "ska_mid/tm_leaf_node/csp_subarray_01"

# Tango host environment variable
TANGO_HOST = "databaseds-tango-base." + namespace + ".svc.cluster.local:10000"

# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIG = os.path.join(DATA_DIR, "mid_telescope/cbf")
CSP_CONFIG = os.path.join(DATA_DIR, "mid_telescope/csp")
TMC_CONFIG = os.path.join(DATA_DIR, "mid_telescope/tmc")
HW_CONFIG = os.path.join(COMMON_CONFIG, "hw_config")
SLIM_CONFIG = os.path.join(COMMON_CONFIG, "slim_config")
CBF_INPUT_DIR = os.path.join(COMMON_CONFIG, "cbf_input_data")
# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]

Next, set the environment arg for TANGO HOST:

In [6]:
print("Will be using HOST: ", TANGO_HOST)
os.environ["TANGO_HOST"] = TANGO_HOST

Will be using HOST:  databaseds-tango-base.ci-ska-mid-psi-1365379550-alexschell.svc.cluster.local:10000


With all the file paths defined we can load in the JSON files and ensure they exist.

In [7]:
print("Getting files...")

INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG, "sys_params/initial_system_param.json")
ASSIGN_RESOURCES_FILE = os.path.join(TMC_CONFIG, "assign_resources.json")
ASSIGN_CSP_RESOURCES_FILE = os.path.join(CSP_CONFIG, "assign_resources.json")
CONFIGURE_SCAN_FILE = os.path.join(TMC_CONFIG, "configure_scan.json")
SCAN_FILE = os.path.join(TMC_CONFIG, "scan.json")
CSP_DELAY_MODEL_FILE = os.path.join(TMC_CONFIG, delay_model_filename)

CBF_INPUT_FILE = f"{CBF_INPUT_DIR}/cbf_input_data.json"
BITE_CONFIG_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/bite_configs.json"
FILTERS_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/filters.json"

DISH_CONFIG_FILE = f"{COMMON_CONFIG}/sys_params/load_dish_config.json"

START_CHANNEL = 0
END_CHANNEL = 14860
START_PORT = 21000

SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    ASSIGN_CSP_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    SCAN_FILE,
    CSP_DELAY_MODEL_FILE,
    CBF_INPUT_FILE,
    BITE_CONFIG_FILE,
    FILTERS_FILE,
    DISH_CONFIG_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

print("Done")

Getting files...
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/sys_params/initial_system_param.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/assign_resources.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/csp/assign_resources.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/csp/assign_resources.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/configure_scan.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/scan.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/tmc/delay_model.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/cbf_input_data/cbf_input_data.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/cbf_input_data/bite_config_parameters/bite_c

Next, set up the hw config to match the boards in use:

In [8]:
if any(i > 4 for i in target_boards_list):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_boards_list = list(map(lambda x: x - 4, target_boards_list))

else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

receptor_ids = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list))

Using standard HW config
HW config: ✔️


If the HW config file exists, we can load it into the pod:

In [9]:
!kubectl cp $HW_CONFIG_FILE $namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

Next we can run the deployer to get everything needed for this run

### 1.3 Create Device Proxies

Using the FQDNs we set earlier, and with the pod spun up, we can create device proxies to the devices we'll be using and check the connection to them.

In [10]:
# CSP Devices
csp_controller = DeviceProxy(CSP_CONTROLLER_FQDN)
print(f"CSP Controller: {csp_controller.Status()}")
csp_subarray = DeviceProxy(CSP_SUBARRAY_FQDN)
print(f"CSP Subarray: {csp_subarray.Status()}")

# TMC Devices
tmc_central_node = DeviceProxy(CENTRAL_NODE_FQDN)
print(f"Central Node: {tmc_central_node.Status()}")
leaf_node = DeviceProxy(LEAF_NODE_FQDN)
print(f"Leaf Node: {leaf_node.Status()}")
leaf_node_subarray = DeviceProxy(LEAF_NODE_SUBARRAY_FQDN)
print(f"Leaf Subarray Node: {leaf_node_subarray.Status()}")
tmc_subarray = DeviceProxy(TMC_SUBARRAY_FQDN)
print(f"TMC subarray Node: {tmc_subarray.Status()}")
delayModelProxy = DeviceProxy(DELAY_MODEL_FQDN)

# Deployer for setup and BITE for data mocking
bite = DeviceProxy(BITE_FQDN)
deployer = DeviceProxy(DEPLOYER_FQDN)

# CBF Device
cbf_controller = DeviceProxy(CBF_CONTROLLER_FQDN)
print(f"CBF controller: {cbf_controller.Status()}")
cbf_subarray = DeviceProxy(CBF_SUBARRAY_FQDN)
print(f"CBF subarray: {cbf_subarray.Status()}")

CSP Controller: The device is in DISABLE state.
CSP Subarray: The device is in DISABLE state.
Central Node: The device is in ON state.
Leaf Node: The device is in ON state.
Leaf Subarray Node: The device is in ON state.
TMC subarray Node: The device is in ON state.
CBF controller: The device is in DISABLE state.
CBF subarray: The device is in DISABLE state.


### 1.4 Downloading Requirements via the Deployer

First, set the board we'll be deploying to and turn on the deployer device.

In [11]:
deployer.On()
deployer.targetTalons = target_boards_list
print(deployer.targetTalons)
deployer.generate_config_jsons()

[2]


Once started and configured, the required devices can then be downloaded.

In [12]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(3000)

Now configure the device database with the downloaded devices:

In [13]:
deployer.configure_db()

## 2 Setting up the Devices

### 2.1 Set up TMC Central Node

Now setup the CSP proxy, and load in the relevant values:

In [14]:
tmc_central_node.adminMode = 0
tmc_central_node.simulationMode = simulation_mode

In [16]:
# Set relevant values on the mid-csp controller
csp_controller.adminMode = 0
csp_subarray.adminMode = 0
sleep(1)
csp_controller.cbfSimulationMode = simulation_mode
csp_subarray.SimulationMode = simulation_mode
sleep(1)
if (
    csp_controller.read_attribute("adminMode").value == 0
    and csp_controller.read_attribute("cbfSimulationMode").value == 0
):
    print("Set to simulation mode off.")
elif (
    csp_controller.read_attribute("adminMode").value == 0
    and csp_controller.read_attribute("cbfSimulationMode").value == 1
):
    print("Set to simulation mode on.")
else:
    print("Error, couldn't set values!")

Set to simulation mode off.


Next we can load in the dish config file to the central node:

In [17]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = f.read()
tmc_central_node.LoadDishCfg(dish_config_json)

[array([2], dtype=int32), ['1720566931.0558825_274913848143126_LoadDishCfg']]

Now, load in the initial parameters file:

In [18]:
with open(INIT_SYS_PARAM_FILE, encoding="utf-8") as init_file:
    data = json.load(init_file)
print("Initial system parameter file:")
print(json.dumps(data, indent=1))

Initial system parameter file:
{
 "interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0",
 "dish_parameters": {
  "SKA001": {
   "vcc": 1,
   "k": 1108
  },
  "SKA036": {
   "vcc": 2,
   "k": 1108
  },
  "SKA063": {
   "vcc": 3,
   "k": 1100
  },
  "SKA100": {
   "vcc": 4,
   "k": 1101
  }
 }
}


In [19]:
# After confirming the file is correct load into the CBF
cbf_controller.InitSysParam(json.dumps(data))

[array([0], dtype=int32), ['CbfController InitSysParam command completed OK']]

### 2.2 Turning on the Telescope via TMC

We can now check if the VCC mapping was loaded

In [20]:
print(leaf_node.sourceDishVccConfig)

{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?ska-sdp-tmlite-repository-1.0.0#tmdata"], "tm_data_filepath": "instrument/ska1_mid_psi/ska-mid-cbf-system-parameters.json"}


In [27]:
print(leaf_node.dishVccConfig)

{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 119}, "SKA036": {"vcc": 2, "k": 1127}, "SKA063": {"vcc": 3, "k": 620}, "SKA100": {"vcc": 4, "k": 101}}}


Now, turn the telescope itself on (TelescopeOn may complain about VCC config, have to wait for it to resolve):

In [28]:
tmc_central_node.set_timeout_millis(100000)
tmc_central_node.TelescopeOn()
sleep(100)

print(tmc_central_node.status())
print(csp_controller.status())

The device is in ON state.
The device is in ON state.


In [29]:
print("Verify device states:")
print(f"\tTMC central node: {tmc_central_node.State()}. Should be ON")
print(f"\tTMC subarray node: {tmc_subarray.State()}. Should be ON")
print(f"\tCSP controller node: {csp_controller.State()}. Should be ON")
print(f"\tCSP subarray node: {csp_subarray.State()}. Should be ON")
print(f"\tCBF subarray node: {cbf_subarray.State()}. Should be ON")

Verify device states:
	TMC central node: ON. Should be ON
	TMC subarray node: ON. Should be ON
	CSP controller node: ON. Should be ON
	CSP subarray node: ON. Should be ON
	CBF subarray node: ON. Should be ON


## 3 Controlling the 5-Point Scan from TMC

### 3.1 Set up JSON Files

Start by getting the base file to use to assign resources to TMC

In [30]:
with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as assign_file:
    basic_assign_json = json.load(assign_file)

Load in the config file for TMC

In [31]:
with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as config_file:
    basic_configure_json = json.load(config_file)

Now overwrite what we'll need to modify in the files:

In [32]:
today = date.today().strftime("%Y%m%d")
random_id = random.randint(0, 99999)
beams = [
    {"beam_id": "vis0", "function": "visibilities"},
    {"beam_id": "pss1", "search_beam_id": 1, "function": "pulsar search"},
    {"beam_id": "pss2", "search_beam_id": 2, "function": "pulsar search"},
    {"beam_id": "pst1", "timing_beam_id": 1, "function": "pulsar timing"},
    {"beam_id": "pst2", "timing_beam_id": 2, "function": "pulsar timing"},
    {"beam_id": "vlbi1", "vlbi_beam_id": 1, "function": "vlbi"},
]

port_list = []
run = 0
while run * 20 + START_CHANNEL <= END_CHANNEL:
    port_list.append([run * 20 + START_CHANNEL, START_PORT + run])
    run += 1

EB_ID = f"eb-testtmc-{today}-{random_id:05d}"
PB_ID = f"pb-testrealtime-{today}-{random_id:05d}"

# Configure the assign resources JSON for TMC
basic_assign_json["sdp"]["processing_blocks"][0]["parameters"]["queue_connector_configuration"][
    "exchanges"
][0]["source"]["servers"] = server
basic_assign_json["dish"]["receptor_ids"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list)
)
basic_assign_json["sdp"]["execution_block"]["beams"] = beams
basic_assign_json["sdp"]["processing_blocks"][0]["pb_id"] = PB_ID
basic_assign_json["sdp"]["execution_block"]["eb_id"] = EB_ID
basic_assign_json["sdp"]["resources"]["receptors"] = list(
    map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list)
)

# Configure the configuration JSON for TMC
basic_configure_json["csp"]["common"]["config_id"] = "1 receptor, band 1, 1 FSP, no options"
basic_configure_json["csp"]["subarray"]["subarray_name"] = "1 receptor"

# Set the FSP ID to match the board and set networking
basic_configure_json["csp"]["cbf"]["fsp"][0]["fsp_id"] = target_boards_list[0]
basic_configure_json["csp"]["cbf"]["fsp"][0]["frequency_slice_id"] = target_boards_list[0]

print("Modified assign json:")
print(json.dumps(basic_assign_json, indent=1))
print("===================")
print(json.dumps(basic_configure_json, indent=1))
print("===================")

Modified assign json:
{
 "interface": "https://schema.skao.int/ska-tmc-assignresources/2.1",
 "transaction_id": "txn-....-00001",
 "subarray_id": 1,
 "dish": {
  "receptor_ids": [
   "SKA036"
  ]
 },
 "sdp": {
  "interface": "https://schema.skao.int/ska-sdp-assignres/0.4",
  "resources": {
   "csp_links": [
    1,
    2,
    3,
    4
   ],
   "receptors": [
    "SKA036"
   ],
   "receive_nodes": 1
  },
  "execution_block": {
   "eb_id": "eb-testtmc-20240709-45507",
   "context": {},
   "max_length": 21600.0,
   "channels": [
    {
     "channels_id": "vis_channels",
     "spectral_windows": [
      {
       "spectral_window_id": "fsp_1_channels",
       "count": 14880,
       "start": 0,
       "stride": 1,
       "freq_min": 300000000.0,
       "freq_max": 550000000.0,
       "link_map": [
        [
         0,
         0
        ],
        [
         200,
         1
        ],
        [
         744,
         2
        ],
        [
         944,
         3
        ]
       ]
      }


### 3.2 Assign Resources to the Telescope and Configure Subarray

Now, we can use TMC to pass the configurations down to the devices used (SDP,CSP...)

In [33]:
tmc_subarray.AssignResources(json.dumps(basic_assign_json))
wait_for_state(tmc_subarray, 2)

 ⣯ Poll# 6: Current state is IDLE, waiting for 2...r 2...
Finished with: IDLE


After this step, check the -sdp namespace to ensure the vis pod is spun up:

In [34]:
# Grab the pod name
!kubectl -n $namespace-sdp get pods | grep vis-receive

proc-pb-testrealtime-20240709-45507-vis-receive-00-0   6/6     Running   0          47s


Now, we can send the configuration to TMC:

In [44]:
tmc_subarray.Configure(json.dumps(basic_configure_json))
wait_for_state(tmc_subarray, 4)

 ⣯ Poll# 702: Current state is IDLE, waiting for 4...

KeyboardInterrupt: 

### 3.3 Setup BITE Data and Generate

For this version of the notebook, BITE is used to generate the data for mocking a scan.

First, load in all the data configs:

In [36]:
with open(BITE_CONFIG_FILE, encoding="utf-8") as f:
    bite_config_data = json.dumps(json.load(f))
    print("BITE configs:\n")
    print(bite_config_data)
bite.load_bite_config_data(bite_config_data)
sleep(1)

with open(FILTERS_FILE, encoding="utf-8") as f:
    filter_data = json.dumps(json.load(f))
    print("Filters:\n")
    print(filter_data)
bite.load_filter_data(filter_data)
sleep(1)

with open(CBF_INPUT_FILE, encoding="utf-8") as f:
    cbf_input_json = json.load(f)["cbf_input_data"][test_id]
    cbf_input_data = json.dumps(cbf_input_json)
    print("CBF Input Data used to generate BITE data:\n")
    print(cbf_input_data)
bite.load_cbf_input_data(cbf_input_data)

BITE configs:

{"bite_configs": {"basic gaussian noise 1": {"description": "4 seconds of basic gaussian noise with no filtering, using the default mean (0) and standard deviation (0.5).", "sources": [{"description": "Noise input, independent between X and Y pols; unique filter shapes for X and Y pols.", "gaussian": {"pol_x": {"seed": 1234, "filter": "filter_all_pass"}, "pol_y": {"seed": 9876, "filter": "filter_all_pass"}}, "pol_coupling_rho": 0.0, "pol_Y_1_sample_delay": false}], "tone_gens": [], "lstv_seconds": 4}, "basic gaussian noise 2": {"description": "4 seconds of basic gaussian noise with no filtering, using the default mean (0) and standard deviation (0.5).", "sources": [{"description": "Noise input, independent between X and Y pols; unique filter shapes for X and Y pols.", "gaussian": {"pol_x": {"seed": 1234, "filter": "filter_all_pass"}, "pol_y": {"seed": 9876, "filter": "filter_all_pass"}}, "pol_coupling_rho": 0.0, "pol_Y_1_sample_delay": false}], "tone_gens": [], "lstv_sec

[array([0], dtype=int32), ['Successfully loaded CBF input data.']]

With these loaded, the BITE data can be generated:

In [37]:
# kubectl logs -n $NS ds-bite-bite-0 -f

bite.set_timeout_millis(240000)
bite.generate_bite_data()
bite.set_timeout_millis(3000)

The BITE pod can be monitored during this to ensure it correctly produces the data.

## 4 Running the 5-Point Scan

Now we can actually run the scans:

### 4.1 Start Replay

Load in the scan file and augment it to match our needs, then use the configure command to pass it in to the CSP.

Once this is done, we can start the LSTV replay, making sure to record the epoch value provided by it

In [38]:
bite.start_lstv_replay()

[array([0], dtype=int32), ['Successfully started LSTV replay.']]

After this command has been sent, the start time can be extracted from the logs:

In [39]:
!kubectl logs -n $namespace ds-bite-bite-0 | grep 'start_utc_time.unix_tai - ska_epoch_tai ='

1|2024-07-09T23:28:18.042Z|INFO|Dummy-1|config_lstv_replay|bite_client.py#533|tango-device:mid_csp_cbf/ec/bite|start_utc_time_offset = start_utc_time.unix_tai - ska_epoch_tai = 773882935.0402379


In [40]:
target_epoch = 773882935.0

With the target epoch ready, we can load it into CSP's delay model.

In [41]:
with open(CSP_DELAY_MODEL_FILE, encoding="utf-8") as f:
    delay_model = json.load(f)
delay_model["start_validity_sec"] = target_epoch
print("Delay model JSON:")
print(json.dumps(delay_model, indent=1))
print("========================")

Delay model JSON:
{
 "interface": "https://schema.skao.int/ska-csp-delaymodel/3.0",
 "start_validity_sec": 773882935.0,
 "cadence_sec": 400.0,
 "validity_period_sec": 400.0,
 "config_id": "sbi-mvp01-20200325-00001-science_A",
 "subarray": 1,
 "receptor_delays": [
  {
   "receptor": "SKA001",
   "xypol_coeffs_ns": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
   ],
   "ypol_offset_ns": 0.0
  }
 ]
}


In [42]:
delayModelProxy.write_attribute("delayModel", json.dumps(delay_model))

### 4.2 Run Scan 1: No Offset

First, set up the vis pod to monitor for packets, running this output on the dev server.

In [ ]:
!echo kubectl exec -n $namespace-sdp -ti $vis_pod -- bash

Once in, install `tcpdump` and run it by copying this command to the bash:

Then have the subarray scan without setting an offset, such that it scans at [0,0].

In [ ]:
long_basic_scan_json = {
    "interface": "https://schema.skao.int/ska-tmc-scan/2.1",
    "transaction_id": "txn-....-00003",
    "scan_id": 1,
}

In [ ]:
tmc_subarray.Scan(json.dumps(long_basic_scan_json))
wait_for_state(cbf_subarray, 5)
print("Central Scan has started, giving it some time to run...")
sleep(100)
tmc_subarray.EndScan()
wait_for_state(cbf_subarray, 4)

Watch the logs to ensure the proper packet lengths, Once this passes, the offset scans can be run.

### 4.3 Run Offset Scans 2-5

**This is currently blocked as reconfiguration causes low-level TANGO memory allocation issues**

Now, loop through the offset scans, using the combos to set the scan json. For each, we configure by passing in JSON, then scan again, letting it run before stopping.

In [ ]:
count = 4
for offset in SCAN_COMBOS:
    print(f"setting offset to: {offset}")

    # Configure for this offset scan
    partial_configure_json = {
        "interface": "https://schema.skao.int/ska-tmc-configure/2.2",
        "transaction_id": f"txn-....-0000{count}",
        "scan_id": count,
        "pointing": {"target": {"ca_offset_arcsec": offset[0], "ie_offset_arcsec": offset[1]}},
        "tmc": {"partial_configuration": True},
    }
    print("Partial Config to load:")
    print(json.dumps(partial_configure_json, indent=1))
    print(".......")
    leaf_node_subarray.Configure(json.dumps(partial_configure_json))
    wait_for_state(cbf_subarray, 4)
    sleep(10)
    # Send the Scan command along with relevant JSON,incrementing the scan ID and transaction ID
    partial_scan_json = {
        "interface": "https://schema.skao.int/ska-tmc-scan/2.1",
        "transaction_id": f"txn-....-0000{count+4}",
        "scan_id": 2 + count,
    }
    leaf_node_subarray.Scan(json.dumps(partial_scan_json))
    wait_for_state(cbf_subarray, 5)
    # let the scan run for a bit...
    sleep(200)
    leaf_node_subarray.EndScan()
    wait_for_state(cbf_subarray, 4)

    count += 1
    print("============================")
print("Done offsets!")

## 5 Cleanup:

In [ ]:
tmc_subarray.GoToIdle()

In [ ]:
tmc_subarray.ReleaseAllResources()

In [ ]:
tmc_subarray.ObsReset()